# ***Install Packages***

---



In [ ]:
!pip install flask flask-cors pyngrok transformers torch

# ***Import Packages***

---



In [2]:
import torch
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from pyngrok import ngrok

# ***Hugging Face Login***

---

**Note:** Hugging Face access token required, you can create account using [Hugging Face](https://huggingface.co/) and understand how to create access token using [Access Tokens](https://huggingface.co/docs/hub/en/security-tokens)

In [3]:
from huggingface_hub import notebook_login
notebook_login()

# ***Load Models and CORS***

---



In [ ]:
app = Flask(__name__)
CORS(app)

sentiment_classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
tokenizer = AutoTokenizer.from_pretrained("deepset/minilm-uncased-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/minilm-uncased-squad2")
summarizer = pipeline("summarization", model="cnicu/t5-small-booksum")


In [6]:
@app.route('/')
def home():
    return "Flask API is running on Google Colab!"

# ***1: Sentiment Classification***

---



In [7]:
@app.route("/sentiment", methods=["POST"])
def analyze_sentiment():
    data = request.json
    print("Received data:", data)
    review = data.get("text", "")
    if not review:
        return jsonify({"error": "No review provided"}), 400
    prediction = sentiment_classifier(review)[0]
    return jsonify({
        "review": review,
        "predicted_sentiment": prediction['label'],
        "confidence": prediction['score']
    })

# ***2: Translation***

---



In [8]:
@app.route('/translate', methods=['POST'])
def translate_text():
    data = request.json
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "No text provided"}), 400
    translated_text = translator(text)[0]['translation_text']
    return jsonify({"original_text": text, "translated_text": translated_text})

# ***3: Extractive QA***

---



In [9]:
@app.route('/qa', methods=['POST'])
def extractive_qa():
    data = request.json
    context = data.get("text", "")
    question = data.get("question", "")
    if not context or not question:
        return jsonify({"error": "Context and question are required"}), 400

    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits) + 1
    answer_span = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_span)
    return jsonify({"question": question, "answer": answer})

# ***4: Summarization***

---



In [14]:
@app.route('/summarize', methods=['POST'])
def summarize_text_api():
    data = request.json
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "No text provided"}), 400
    outputs = summarizer(text, max_length=53)
    summarized_text = outputs[0]['summary_text']
    return jsonify({"original_text": text, "summarized_text": summarized_text})


# ***ngrok Authentication***

---


**Note:** ngrok account authtoken is required, you can create account using [ngork](https://ngrok.com/) and get authtoken on the account dashboard

In [10]:
!ngrok authtoken <YOUR AUTHTOKEN>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
ngrok.set_auth_token("YOUR AUTHTOKEN")

port = 5000
public_url = ngrok.connect(port).public_url
print(f"🔗 Public URL: {public_url}")

app.run(port=port)